<a href="https://colab.research.google.com/github/AndreRigueira/Alura_RigsMed/blob/main/Alura_Projeto_Busca_Medicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

# Criar a variável de ambiente, usando a API KEY que já deve ter sido importada e habilitada no "Secrets"
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework ADK de agentes do Google
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Médicos e Clínicas --- #
##########################################
def agente_buscador(localidade, especialidade):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para identificar médicos e clínicas na cidade e na especialidade indicadas.
        Foque em no mínimo 5 e no máximo 15 lançamentos relevantes, com base nas avaliações, caso hajam, e no entusiasmo dessas avaliações.
        Se um médico ou clínica tiver poucas reações entusiasmadas ou tiver mais avaliações negativas do que positivas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançmentos relevantes devem ser atuais, de no máximo um ano antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Localidade: {localidade}\nEspecialidade Médica: {especialidade}"

    # Executa o agente
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
############################################################
# --- Agente 2: Avaliador dos Especilistas localizados --- #
############################################################
def agente_avaliador(especialidade, lancamentos):
    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Avaliador
        instruction="""
        Você é um avaliador de reputação, especialista em medicina. Com base na lista de
        lançamentos mais recentes e relevantes buscados, você deve:
        Usar a ferramenta de busca do Google (google_search) para definir a reputação de cada
        especialista (médico ou clínica). Você também pode usar o (google_search) para encontrar mais
        informações sobre esse profissionais e estabelecimentos.
        Ao final, você irá escolher os médicos e estabelecimentos melhor avaliados e com melhor
        reputação.
        """,
        description="Agente que avalia as indicações",
        tools=[google_search]
    )

    entrada_do_agente_avaliador = f"Especialidade:{especialidade}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    lista_indicados = call_agent(avaliador, entrada_do_agente_avaliador)
    return lista_indicados

In [ ]:
##########################################
# --- Agente 3: Redator da Indicação --- #
##########################################
def agente_redator(especialidade, lista_indicados):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é uma Secretária especializada em criar documentos e listas.
            Utilize as indicações fornecidas na Lista de indicados, com base nisso,
            faça uma lista com o nome do Médico ou Estabelecimento, um sucinta avaliação, o telefone de contato, e o link do website onde foi encontrada a informação via (google_search).
            A lista deve ser organizada e de fácil entendimento.
            """,
        description="Agente redator da lista de Indicações"
    )
    entrada_do_agente_redator = f"Especilidade: {especialidade}\nLista de Indicados: {lista_indicados}"
    # Executa o agente
    lista = call_agent(redator, entrada_do_agente_redator)
    return lista

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🩺 Iniciando o Sistema de Indicação de Especialistas Médicos ﮩ٨ـﮩﮩ٨ـ♡ﮩ٨ـﮩﮩ٨ـ")

# --- Obter o Tópico do Usuário ---
localidade = input("❓ Por favor, digite a LOCALIDADE sobre o qual você quer indicações: ")

# Inserir lógica do sistema de agentes
if not localidade:
  print("Por favor, insira uma localidade.")
  localidade = input("❓ Por favor, digite a LOCALIDADE sobre o qual você quer indicações: ")
else:
  especialidade = input("❓ Por favor, digite a ESPECIALIDADE médica sobre o qual você quer indicações: ")
  if not especialidade:
    print("Por favor, insira uma especialidade médica.")
    especialidade = input("❓ Por favor, digite a ESPECIALIDADE médica sobre o qual você quer indicações: ")
  else:
    print(f"Perfeito! Vamos então criar uma Lista de indicações de {especialidade} em {localidade}!")

  # Incluindo o agente buscador
  lancamentos_buscados = agente_buscador(localidade, especialidade)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("-------------------------------------------------------------------")

 # Incluindo o agente avaliador
  lista_indicados = agente_avaliador(especialidade, lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Avaliador) ---\n")
  display(to_markdown(lista_indicados))
  print("-------------------------------------------------------------------")

# Incluindo o Agente Redator da Lista
  rascunho_da_lista = agente_redator(especialidade, lista_indicados)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_da_lista))
  print("-------------------------------------------------------------------")

🩺 Iniciando o Sistema de Indicação de Especialistas Médicos ﮩ٨ـﮩﮩ٨ـ♡ﮩ٨ـﮩﮩ٨ـ
❓ Por favor, digite a LOCALIDADE sobre o qual você quer indicações: Brasília
❓ Por favor, digite a ESPECIALIDADE médica sobre o qual você quer indicações: Oftalmologista
Perfeito! Vamos então criar uma Lista de indicações de Oftalmologista em Brasília!

--- Resultado do Agente 1 (Buscador) ---



> Ok, estou procurando por oftalmologistas em Brasília. Aqui estão algumas opções que encontrei, levando em conta avaliações, entusiasmo e atualidade:
> 
> 
> Com base nas minhas buscas, aqui estão alguns oftalmologistas e clínicas em Brasília que parecem promissores, com ênfase em avaliações positivas e informações recentes:
> 
> 1.  **Mondevi Hospital Oftalmológico:** Considerado por alguns pacientes como a "melhor clínica oftalmológica de Brasília". Eles oferecem uma variedade de serviços, incluindo cirurgia refrativa, glaucoma, catarata e ceratocone. A clínica é bem equipada com tecnologia moderna e tem um corpo clínico qualificado. O Dr. Mário Jampaulo é particularmente recomendado.
> 2.  **Dr. Anderson Braz:** Oftalmologista com muitas avaliações positivas no Doctoralia. Ele tem experiência em cirurgia refrativa, catarata e cirurgia oftalmológica.
> 3.  **Dr. Henrique Jorge de Paula:** Também possui muitas avaliações positivas no Doctoralia. Ele atende na Clínica Pacini de Oftalmologia.
> 4.  **Clínica Pacini de Oftalmologia:** Além do Dr. Henrique, a clínica como um todo parece ser uma boa opção.
> 5.  **Mirar Oftalmologia:** Uma clínica particular que foca em um tratamento médico personalizado e completo. Eles oferecem oftalmologia geral, cirurgia refrativa e tratamento para ceratocone.
> 6.  **Dr. Hamilton Moreira:** Diretor clínico do Médicos de Olhos S.A.. Participou do CXLIX FACO CLUBE BRASÍLIA, um evento importante na área de oftalmologia. Tem experiência em cirurgia facorefrativa.
> 7.  **Hospital Oftalmológico de Brasília:** Um dos locais onde o CXLIX FACO CLUBE BRASÍLIA foi realizado.
> 8. **Pacini Hospital Oftalmológico:** O hospital apoia projetos sociais. Possui unidade em Taguatinga.
> 
> **Eventos e Congressos:**
> 
> *   **68° Congresso Brasileiro de Oftalmologia (CBO 2024):** Realizado em Brasília em setembro de 2024, é um evento de grande prestígio na área.
> *   **CXLIX FACO CLUBE BRASÍLIA:** Um evento de destaque que reúne profissionais para discutir avanços em cirurgias oculares.
> 
> **Observações:**
> 
> *   As avaliações são um bom indicador, mas é sempre recomendável verificar a experiência específica do médico com o seu problema.
> *   Consulte os sites ou entre em contato com as clínicas para obter informações mais detalhadas sobre os serviços oferecidos e os convênios aceitos.
> *   Verifique se os médicos participam de eventos e congressos na área, o que pode indicar um compromisso com a atualização e o aprimoramento profissional.


-------------------------------------------------------------------

--- Resultado do Agente 2 (Avaliador) ---



> Para avaliar a reputação dos oftalmologistas e clínicas em Brasília mencionados, usarei a busca do Google para encontrar informações adicionais, avaliações e notícias relevantes sobre cada um. Isso me ajudará a determinar quais profissionais e estabelecimentos são mais bem avaliados e têm melhor reputação.
> 
> 
> Com base nas informações encontradas, aqui está uma avaliação da reputação dos oftalmologistas e clínicas em Brasília:
> 
> **Oftalmologistas e Clínicas:**
> 
> 1.  **Mondevi Hospital Oftalmológico:** Considerada por alguns pacientes como a "melhor clínica oftalmológica de Brasília" \[12, 27, 28]. Possui avaliações positivas e é bem equipada com tecnologia moderna e um corpo clínico qualificado. O Dr. Mário Jampaulo é particularmente recomendado \[8]. A clínica possui uma avaliação média de 4,7 de 5,0 estrelas no Google \[12].
> 2.  **Dr. Anderson Braz:** Oftalmologista com muitas avaliações positivas no Doctoralia \[2, 3, 4, 5, 6]. É especialista em cirurgia refrativa e córnea, com experiência de anos \[5]. Considerado o oftalmologista mais bem avaliado de Brasília, com centenas de avaliações 5 estrelas no Google \[5, 6].
> 3.  **Dr. Henrique Jorge de Paula:** Possui muitas avaliações positivas no Doctoralia \[4, 7, 24, 25]. Atende na Clínica Pacini de Oftalmologia \[4].
> 4.  **Clínica Pacini de Oftalmologia:** A clínica como um todo parece ser uma boa opção, com avaliações positivas no Doctoralia \[4, 10]. Possui uma equipe médica excelente e equipamentos modernos \[10].
> 5.  **Mirar Oftalmologia:** Clínica particular que foca em um tratamento médico personalizado e completo \[15, 17]. Oferece oftalmologia geral, cirurgia refrativa e tratamento para ceratocone \[17].
> 6.  **Dr. Hamilton Moreira:** Diretor clínico do Médicos de Olhos S.A. \[9, 11, 31, 38]. Participou do CXLIX FACO CLUBE BRASÍLIA e tem experiência em cirurgia facorefrativa \[9]. Foi eleito o novo presidente do Conselho Brasileiro de Oftalmologia (CBO) \[40].
> 7.  **Médicos de Olhos S.A.:** Está entre as maiores redes de clínicas do país, com 16 unidades \[18]. Possui um aplicativo para agendamento de consultas e exames \[37]. A empresa busca inovação para estreitar o relacionamento com seus pacientes \[18, 26, 31].
> 8.  **Hospital Oftalmológico de Brasília (HOB):** Há referências sobre ele, mas sem muitos detalhes nas buscas \[13, 20].
> 9.  **Pacini Hospital Oftalmológico:** Possui unidades na Asa Sul e Taguatinga \[8, 16, 35]. Oferece uma ampla variedade de exames e cirurgias \[23, 39]. O hospital apoia projetos sociais e tem responsabilidade social \[8, 33, 35].
> 
> **Considerações Finais:**
> 
> *   **Melhor Reputação:** Com base nas informações disponíveis, o **Dr. Anderson Braz** e o **Mondevi Hospital Oftalmológico** parecem ter a melhor reputação, devido às avaliações positivas e à qualidade dos serviços oferecidos. A **Clínica Pacini de Oftalmologia** e o **Pacini Hospital Oftalmológico** também se destacam pela tradição e variedade de serviços.
> *   **Destaque:** O **Dr. Hamilton Moreira** é um nome importante na área, com experiência e participação em eventos relevantes, além de ocupar um cargo de direção no Médicos de Olhos S.A.
> *   **Recomendações:** É sempre recomendável verificar a experiência específica do médico com o seu problema e consultar os sites ou entrar em contato com as clínicas para obter informações mais detalhadas.
> 
> **Observações:**
> 
> *   As avaliações são um bom indicador, mas é sempre recomendável verificar a experiência específica do médico com o seu problema.
> *   Consulte os sites ou entre em contato com as clínicas para obter informações mais detalhadas sobre os serviços oferecidos e os convênios aceitos.
> *   Verifique se os médicos participam de eventos e congressos na área, o que pode indicar um compromisso com a atualização e o aprimoramento profissional.
> 


-------------------------------------------------------------------

--- Resultado do Agente 3 (Redator) ---



> ## Lista de Indicações - Oftalmologistas e Clínicas em Brasília
> 
> **1. Mondevi Hospital Oftalmológico**
> 
> *   **Avaliação:** Considerada por alguns pacientes como a "melhor clínica oftalmológica de Brasília". Bem equipada e com corpo clínico qualificado.
> *   **Telefone:** Não encontrado diretamente na pesquisa. Recomenda-se buscar no site.
> *   **Website:** Não listado diretamente. Buscar por "Mondevi Hospital Oftalmológico Brasília" no Google.
> 
> **2. Dr. Anderson Braz**
> 
> *   **Avaliação:** Oftalmologista com muitas avaliações positivas, especialista em cirurgia refrativa e córnea. Considerado um dos mais bem avaliados de Brasília.
> *   **Telefone:** Não encontrado diretamente na pesquisa. Recomenda-se buscar no Doctoralia.
> *   **Website:** Não listado diretamente. Buscar por "Dr. Anderson Braz Oftalmologista Brasília" no Google ou Doctoralia.
> 
> **3. Dr. Henrique Jorge de Paula**
> 
> *   **Avaliação:** Possui muitas avaliações positivas no Doctoralia. Atende na Clínica Pacini de Oftalmologia.
> *   **Telefone:** Não encontrado diretamente na pesquisa. Recomenda-se buscar no Doctoralia.
> *   **Website:** Não listado diretamente. Buscar por "Dr. Henrique Jorge de Paula Oftalmologista Brasília" no Google ou Doctoralia, ou verificar o site da Clínica Pacini.
> 
> **4. Clínica Pacini de Oftalmologia**
> 
> *   **Avaliação:** Clínica com avaliações positivas, equipe médica excelente e equipamentos modernos.
> *   **Telefone:** Não encontrado diretamente na pesquisa. Recomenda-se buscar no Google.
> *   **Website:** Não listado diretamente. Buscar por "Clínica Pacini Oftalmologia Brasília" no Google.
> 
> **5. Mirar Oftalmologia**
> 
> *   **Avaliação:** Clínica particular com foco em tratamento médico personalizado e completo.
> *   **Telefone:** Não encontrado diretamente na pesquisa. Recomenda-se buscar no Google.
> *   **Website:** Não listado diretamente. Buscar por "Mirar Oftalmologia Brasília" no Google.
> 
> **6. Dr. Hamilton Moreira**
> 
> *   **Avaliação:** Diretor clínico do Médicos de Olhos S.A., experiente em cirurgia facorefrativa e eleito presidente do Conselho Brasileiro de Oftalmologia (CBO).
> *   **Telefone:** Não encontrado diretamente na pesquisa. Recomenda-se buscar no site do Médicos de Olhos S.A.
> *   **Website:** Não listado diretamente. Buscar por "Médicos de Olhos S.A." no Google.
> 
> **7. Médicos de Olhos S.A.**
> 
> *   **Avaliação:** Uma das maiores redes de clínicas do país, com várias unidades.
> *   **Telefone:** Não encontrado diretamente na pesquisa. Recomenda-se buscar no site.
> *   **Website:** Não listado diretamente. Buscar por "Médicos de Olhos S.A." no Google.
> 
> **8. Hospital Oftalmológico de Brasília (HOB)**
> 
> *   **Avaliação:** Há referências, mas sem muitos detalhes nas buscas.
> *   **Telefone:** Não encontrado diretamente na pesquisa. Recomenda-se buscar no Google.
> *   **Website:** Não listado diretamente. Buscar por "Hospital Oftalmológico de Brasília (HOB)" no Google.
> 
> **9. Pacini Hospital Oftalmológico**
> 
> *   **Avaliação:** Possui unidades na Asa Sul e Taguatinga, oferece ampla variedade de exames e cirurgias, e apoia projetos sociais.
> *   **Telefone:** Não encontrado diretamente na pesquisa. Recomenda-se buscar no Google.
> *   **Website:** Não listado diretamente. Buscar por "Pacini Hospital Oftalmológico Brasília" no Google.
> 


-------------------------------------------------------------------
